In [ ]:
## import pandas as pd
import numpy as np
import tensorflow as tf
import os 
from glob import glob
import tqdm
from tqdm import tqdm
import pandas as pd
import pydicom
import nrrd
from einops import rearrange, reduce, repeat

In [ ]:
import cv2
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import math
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from sklearn.model_selection import train_test_split

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"

# 내가 사용할 GPU번호(0,1,2,3)
os.environ["CUDA_VISIBLE_DEVICES"]="1"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus: 
    try: # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus: 
            tf.config.experimental.set_memory_growth(gpu, True)
             
        logical_gpus = tf.config.experimental.list_logical_devices('GPU') 
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs") 
    
    except RuntimeError as e: # Memory growth must be set before GPUs have been initialized 
        print(e)

In [ ]:
ptid = sorted(os.listdir('./data/original1125/'))
df_ptid = pd.DataFrame(ptid, columns=['ptid'])
all_inputpath = list('./data/original1125/'+df_ptid['ptid']+'/img/')
all_label = './data/original1125/'+df_ptid['ptid']
all_labelpath = []
for i in range(len(all_label)):
    all_labelpath.append(glob(all_label[i]+'/*.nrrd')[0])

In [ ]:
train_input_files = all_inputpath[:31]
train_label_files = all_labelpath[:31]

val_input_files = all_inputpath[31:38]
val_label_files = all_labelpath[31:38]

test_input_files = all_inputpath[38:]
test_label_files = all_labelpath[38:]

print(len(train_input_files), len(val_input_files), len(test_input_files))

In [ ]:
BATCH_SIZE = 8
img_size = 128
weights = None
# weights = 'imagenet'
learning_rate = 1e-4
EPOCHS = 400
dropout_rate = 0.1

In [ ]:
train_inp_files = glob(f'./data{img_size}/train_input_img_{img_size}/*.npy')
train_targ_files = glob(f'./data{img_size}/train_label_img_{img_size}/*.npy')

val_inp_files = glob(f'./data{img_size}/val_input_img_{img_size}/*.npy')
val_targ_files = glob(f'./data{img_size}/val_label_img_{img_size}/*.npy')

In [ ]:
train_inp_files = glob(f'./data{img_size}/train_input_img_{img_size}/*.npy')
train_targ_files = glob(f'./data{img_size}/train_label_img_{img_size}/*.npy')

val_inp_files = glob(f'./data{img_size}/val_input_img_{img_size}/*.npy')
val_targ_files = glob(f'./data{img_size}/val_label_img_{img_size}/*.npy')

In [ ]:
train_inp_files, train_targ_files = shuffle(train_inp_files, train_targ_files, random_state=42)
len(train_inp_files), len(val_inp_files), len(train_targ_files), len(val_targ_files)

In [ ]:
def train_map_func(inp_path, targ_path):
    inp = np.load(inp_path)
    inp = inp.astype(np.float32)
    targ = np.load(targ_path)
    targ = targ.astype(np.float32)
    inp, targ = augmentation(inp, targ)
    
    return inp, targ

def val_map_func(inp_path, targ_path):
    inp = np.load(inp_path)
    inp = inp.astype(np.float32)
    targ = np.load(targ_path)
    targ = targ.astype(np.float32)
    return inp, targ

def augmentation(inp, targ):
    inp, targ = random_rot(inp, targ)
    inp, targ = random_flip(inp, targ)
    
    return inp, targ

def random_rot(inp, targ):
    k = np.random.randint(4)
    inp = np.rot90(inp, k)
    targ = np.rot90(targ, k)
    
    return inp, targ

def random_flip(inp, targ):
    f = np.random.randint(2)
    if f == 0:
        inp = np.fliplr(inp)
        targ = np.fliplr(targ)
        
    return inp, targ

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_inp_files, train_targ_files))
train_dataset = train_dataset.map(lambda item1, item2: tf.numpy_function(train_map_func, [item1, item2], [tf.float32, tf.float32]), num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_inp_files, val_targ_files))
val_dataset = val_dataset.map(lambda item1, item2: tf.numpy_function(val_map_func, [item1, item2], [tf.float32, tf.float32]), num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
from tensorflow.keras.utils import Sequence
class Dataloader(Sequence):

    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.on_epoch_end()

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

		# batch 단위로 직접 묶어줘야 함
    def __getitem__(self, idx):
				# sampler의 역할(index를 batch_size만큼 sampling해줌)
        # indices = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]
        # batch_x = [self.x[i] for i in indices]
        # batch_y = [self.y[i] for i in indices]

        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        return np.array(batch_x), np.array(batch_y)


train_loader = Dataloader(X_train, y_train, 128)
valid_loader = Dataloader(X_valid, y_valid, 128)
# test_loader = Dataloader(x, y, 128)

history = model.fit(train_loader, epochs=300, 
                    validation_data=valid_loader,
                    callbacks=[cb]
                    )

In [ ]:
model = tf.keras.models.load_model('models/3dunet_dice_128.h5', compile=False)

In [ ]:
def predict(img_paths, stride=128): #batch_size
    results = []
    for img_path in img_paths:
        l = len(os.listdir(img_path))
        batch_size = int(np.ceil(l//stride))
        rawimg = np.zeros((512,512,l))
        for i in range(l):
            slice = pydicom.read_file(img_path+'IMG'+str(i+1))
            s = int(slice.RescaleSlope)
            b = int(slice.RescaleIntercept)
            image = s * slice.pixel_array + b
            rawimg[:,:,i]=image
        rawimg = np.expand_dims(rawimg,axis=3)
        clipimg = np.clip(rawimg, -1450,50)
        img = (clipimg-np.min(clipimg))/(np.max(clipimg)-np.min(clipimg))
        img = img.astype(np.float32)
        crop = []
        position = []
        batch_count = 0

        result_img = np.zeros_like(img)
        # print(result_img.shape)
        voting_mask = np.zeros_like(img)

        for top in tqdm(range(0, img.shape[0], stride)):
            for left in range(0, img.shape[1], stride):
                for slicer in range(0, img.shape[2], stride):
                    piece = np.zeros([img_size, img_size, img_size,1], np.float32)
                    temp = img[top:top+img_size, left:left+img_size, slicer:slicer+img_size, :]
                    piece[:temp.shape[0], :temp.shape[1], :temp.shape[2], :] = temp
                    crop.append(piece)
                    position.append([top, left, slicer])
                    batch_count += 1
                    if batch_count == batch_size:
                        crop = np.array(crop)
                        pred = model(crop)
                        crop = []
                        batch_count = 0
                        for num, (t, l, sl) in enumerate(position):
                            piece = pred[num]
                            h, w, s, c= result_img[t:t+img_size, l:l+img_size, sl:sl+img_size, :].shape
                            result_img[t:t+img_size, l:l+img_size, sl:sl+img_size, :] += piece[:h, :w, :s, :c]
                            voting_mask[t:t+img_size, l:l+img_size, sl:sl+img_size, :] += 1
                        position = []
        
        result_img = result_img/voting_mask
        result_img = result_img.astype(np.float32)
        results.append(result_img)
        
    return results

In [ ]:
result = predict(val_input_files[:2], stride=128)

In [ ]:
print(np.array(result[0]).shape)

print(np.array(result[1]).shape)

In [ ]:
rr = np.array(result[1])
cnt =1
# plt.subplots_adjust(0,0,1,1,0.01,0.01)
plt.figure(figsize=(16,11))
for i in range(250,260):    
    plt.subplot(3,4,cnt)
    plt.imshow(np.squeeze(rr[:,:,i]),'gray')
    plt.axis('off')
    plt.tight_layout()
    cnt+=1

In [ ]:
readdata, header = nrrd.read(val_label_files[1], index_order='C') #readdata (c h w)
readdata = readdata[::-1] #reverse axial (-c h w)
readdata = rearrange(readdata, 'c h w  -> h w c')
readdata[readdata>0.5] = 1 #integrate label

In [ ]:
readdata.shape

In [ ]:
cnt =1
# plt.subplots_adjust(0,0,1,1,0.01,0.01)
plt.figure(figsize=(16,11))
for i in range(250,260):    
    plt.subplot(3,4,cnt)
    plt.imshow(readdata[:,:,i],'gray')
    plt.axis('off')
    plt.tight_layout()
    cnt+=1

In [ ]:
rr = np.array(result[1])
cnt =1
# plt.subplots_adjust(0,0,1,1,0.01,0.01)
plt.figure(figsize=(16,11))
for i in range(250,260):    
    plt.subplot(3,4,cnt)
    plt.imshow(np.squeeze(rr[:,:,i]),'gray')
    plt.axis('off')
    plt.tight_layout()
    cnt+=1

In [ ]:
rawimg.shape

In [ ]:
test_result = predict(test_input_files[:1], stride=128)

In [ ]:
np.array(test_result).shape

In [ ]:
n = 0
label_img, header = nrrd.read(test_label_files[n], index_order='C') #readdata (c h w)
label_img = label_img[::-1] #reverse axial (-c h w)
label_img = rearrange(label_img, 'c h w  -> h w c')
label_img[label_img>0.5] = 1 #integrate label

l = len(os.listdir(test_input_files[n]))
rawimg = np.zeros((512,512,l))
for i in range(l):
    slice = pydicom.read_file(test_input_files[n]+'IMG'+str(i+1))
    s = int(slice.RescaleSlope)
    b = int(slice.RescaleIntercept)
    image = s * slice.pixel_array + b
    rawimg[:,:,i]=image
rawimg = (rawimg-np.min(rawimg))/(np.max(rawimg)-np.min(rawimg))

rr = np.array(test_result[0])   

In [ ]:
rr.shape, label_img.shape

In [ ]:
for i in range(0,rr.shape[2]):
    print(i)
    plt.figure(figsize=(20,10))
    plt.subplot(1,3,1)
    plt.imshow(rawimg[:,:,i], cmap='gray')
    plt.axis('off')
    plt.subplot(1,3,2)
    plt.imshow(label_img[:,:,i], cmap='gray')
    plt.axis('off')
    plt.subplot(1,3,3)
    plt.imshow(np.squeeze(rr[:,:,i]), cmap='gray')
    plt.axis('off')
    # plt.savefig('result'+str(i)+'.png')
    plt.show()

In [ ]:
sample_test_result = np.squeeze(rr)
np.save('./images/sampletest0.npy',sample_test_result)

In [ ]:
np.save('./images/samplelabel0.npy',label_img)

In [ ]:
n = 1
label_img, header = nrrd.read(val_label_files[n], index_order='C') #readdata (c h w)
label_img = label_img[::-1] #reverse axial (-c h w)
label_img = rearrange(label_img, 'c h w  -> h w c')
label_img[label_img>0.5] = 1 #integrate label

l = len(os.listdir(val_input_files[n]))
rawimg = np.zeros((512,512,l))
for i in range(l):
    slice = pydicom.read_file(val_input_files[n]+'IMG'+str(i+1))
    s = int(slice.RescaleSlope)
    b = int(slice.RescaleIntercept)
    image = s * slice.pixel_array + b
    rawimg[:,:,i]=image
rawimg = (rawimg-np.min(rawimg))/(np.max(rawimg)-np.min(rawimg))

rr = np.array(result[1])   